In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install required packages
!pip install wandb torch torchvision pandas numpy matplotlib seaborn scikit-learn mlflow

# Set up Kaggle API
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
# Upload your kaggle.json to Colab and run:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
# Download the dataset
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
!unzip -q walmart-recruiting-store-sales-forecasting.zip

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 711MB/s]


In [5]:
!unzip -q train.csv.zip
!unzip -q stores.csv.zip
!unzip -q test.csv.zip
!unzip -q features.csv.zip

unzip:  cannot find or open stores.csv.zip, stores.csv.zip.zip or stores.csv.zip.ZIP.


In [17]:
# model_experiment_nbeats_enhanced.ipynb

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import mlflow
import mlflow.pytorch
import wandb
import os
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import dagshub

# DagsHub and MLflow Setup
def setup_tracking(project_name="walmart-nbeats-forecasting", dagshub_repo_owner="ekvirika", dagshub_repo_name="WalmartRecruiting"):
    """
    Setup MLflow, DagsHub, and Wandb tracking

    Args:
        project_name: Name for the experiment
        dagshub_repo_owner: Your DagsHub username
        dagshub_repo_name: Your DagsHub repository name
    """

    # DagsHub Setup
    dagshub.init(repo_owner=dagshub_repo_owner, repo_name=dagshub_repo_name, mlflow=True)

    # Set MLflow tracking URI to DagsHub
    os.environ['MLFLOW_TRACKING_URI'] = f'https://dagshub.com/{dagshub_repo_owner}/{dagshub_repo_name}.mlflow'
    os.environ['MLFLOW_TRACKING_USERNAME'] = dagshub.auth.get_username()
    os.environ['MLFLOW_TRACKING_PASSWORD'] = dagshub.auth.get_password()

    # Initialize Wandb
    wandb.init(
        project=project_name,
        config={
            "framework": "pytorch",
            "model": "nbeats",
            "dataset": "walmart_sales"
        }
    )

    print("✅ Tracking systems initialized:")
    print(f"   - DagsHub: https://dagshub.com/{dagshub_repo_owner}/{dagshub_repo_name}")
    print(f"   - MLflow: {os.environ.get('MLFLOW_TRACKING_URI')}")
    print(f"   - Wandb: {wandb.run.get_url()}")

class NBEATSBlock(nn.Module):
    def __init__(self, input_size, theta_size, basis_function, layers, layer_size):
        super().__init__()
        self.layers = nn.ModuleList([nn.Linear(input_size, layer_size)] +
                                    [nn.Linear(layer_size, layer_size) for _ in range(layers-1)])
        self.basis_function = basis_function
        self.backcast_fc = nn.Linear(layer_size, theta_size)
        self.forecast_fc = nn.Linear(layer_size, theta_size)

    def forward(self, x):
        # Forward through fully connected layers
        for layer in self.layers:
            x = torch.relu(layer(x))

        # Generate theta parameters
        backcast_theta = self.backcast_fc(x)
        forecast_theta = self.forecast_fc(x)

        # Apply basis functions
        backcast, forecast = self.basis_function(backcast_theta, forecast_theta)

        return backcast, forecast

class GenericBasis(nn.Module):
    def __init__(self, backcast_size, forecast_size):
        super().__init__()
        self.backcast_size = backcast_size
        self.forecast_size = forecast_size

    def forward(self, backcast_theta, forecast_theta):
        # Generic basis function - fixed size handling
        batch_size = backcast_theta.shape[0]
        backcast = backcast_theta[:, :self.backcast_size]
        forecast = forecast_theta[:, :self.forecast_size]

        # Ensure correct dimensions
        if backcast.shape[1] < self.backcast_size:
            padding = torch.zeros(batch_size, self.backcast_size - backcast.shape[1], device=backcast.device)
            backcast = torch.cat([backcast, padding], dim=1)

        if forecast.shape[1] < self.forecast_size:
            padding = torch.zeros(batch_size, self.forecast_size - forecast.shape[1], device=forecast.device)
            forecast = torch.cat([forecast, padding], dim=1)

        return backcast, forecast

class SeasonalityBasis(nn.Module):
    def __init__(self, harmonics, backcast_size, forecast_size):
        super().__init__()
        self.harmonics = harmonics
        self.backcast_size = backcast_size
        self.forecast_size = forecast_size

    def forward(self, backcast_theta, forecast_theta):
        # Seasonality basis using Fourier series
        batch_size = backcast_theta.shape[0]
        device = backcast_theta.device

        # Create time indices on the correct device
        backcast_time = torch.linspace(0, 1, self.backcast_size, device=device).unsqueeze(0).repeat(batch_size, 1)
        forecast_time = torch.linspace(0, 1, self.forecast_size, device=device).unsqueeze(0).repeat(batch_size, 1)

        backcast = torch.zeros_like(backcast_time)
        forecast = torch.zeros_like(forecast_time)

        for i in range(self.harmonics):
            cos_coef = backcast_theta[:, i].unsqueeze(-1)
            sin_coef = backcast_theta[:, i + self.harmonics].unsqueeze(-1)

            # Apply Fourier basis
            backcast += (cos_coef * torch.cos(2 * np.pi * (i + 1) * backcast_time) +
                        sin_coef * torch.sin(2 * np.pi * (i + 1) * backcast_time))

            cos_coef_f = forecast_theta[:, i].unsqueeze(-1)
            sin_coef_f = forecast_theta[:, i + self.harmonics].unsqueeze(-1)

            forecast += (cos_coef_f * torch.cos(2 * np.pi * (i + 1) * forecast_time) +
                        sin_coef_f * torch.sin(2 * np.pi * (i + 1) * forecast_time))

        return backcast, forecast

class TrendBasis(nn.Module):
    def __init__(self, degree_of_polynomial, backcast_size, forecast_size):
        super().__init__()
        self.degree = degree_of_polynomial
        self.backcast_size = backcast_size
        self.forecast_size = forecast_size

    def forward(self, backcast_theta, forecast_theta):
        # Polynomial trend basis
        batch_size = backcast_theta.shape[0]
        device = backcast_theta.device

        backcast_time = torch.linspace(-1, 1, self.backcast_size, device=device).unsqueeze(0).repeat(batch_size, 1)
        forecast_time = torch.linspace(-1, 1, self.forecast_size, device=device).unsqueeze(0).repeat(batch_size, 1)

        backcast = torch.zeros_like(backcast_time)
        forecast = torch.zeros_like(forecast_time)

        for i in range(self.degree + 1):
            backcast += backcast_theta[:, i].unsqueeze(-1) * (backcast_time ** i)
            forecast += forecast_theta[:, i].unsqueeze(-1) * (forecast_time ** i)

        return backcast, forecast

class NBEATS(nn.Module):
    def __init__(self, backcast_length, forecast_length,
                 stack_types=['generic', 'seasonality', 'trend'],
                 nb_blocks_per_stack=3, hidden_layer_units=128,
                 nb_harmonics=10, polynomial_degree=3):
        super().__init__()
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length
        self.hidden_layer_units = hidden_layer_units
        self.nb_blocks_per_stack = nb_blocks_per_stack
        self.stack_types = stack_types

        self.stacks = nn.ModuleList()

        for stack_type in stack_types:
            stack_blocks = nn.ModuleList()

            for _ in range(nb_blocks_per_stack):
                if stack_type == 'generic':
                    theta_size = backcast_length + forecast_length
                    basis_function = GenericBasis(backcast_length, forecast_length)
                elif stack_type == 'seasonality':
                    theta_size = 2 * nb_harmonics
                    basis_function = SeasonalityBasis(nb_harmonics, backcast_length, forecast_length)
                elif stack_type == 'trend':
                    theta_size = polynomial_degree + 1
                    basis_function = TrendBasis(polynomial_degree, backcast_length, forecast_length)

                block = NBEATSBlock(
                    input_size=backcast_length,
                    theta_size=theta_size,
                    basis_function=basis_function,
                    layers=4,
                    layer_size=hidden_layer_units
                )
                stack_blocks.append(block)

            self.stacks.append(stack_blocks)

    def forward(self, backcast):
        # Initialize forecast with correct device
        forecast = torch.zeros(backcast.size(0), self.forecast_length, device=backcast.device)

        for stack in self.stacks:
            for block in stack:
                b, f = block(backcast)
                backcast = backcast - b
                forecast = forecast + f

        return forecast

class WalmartDataset(Dataset):
    def __init__(self, data, backcast_length, forecast_length):
        self.data = data
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length

    def __len__(self):
        return len(self.data) - self.backcast_length - self.forecast_length + 1

    def __getitem__(self, idx):
        backcast = self.data[idx:idx + self.backcast_length]
        forecast = self.data[idx + self.backcast_length:idx + self.backcast_length + self.forecast_length]
        return torch.FloatTensor(backcast), torch.FloatTensor(forecast)

def calculate_metrics(actuals, predictions):
    """Calculate comprehensive evaluation metrics"""
    actuals = np.array(actuals)
    predictions = np.array(predictions)

    # Basic metrics
    mae = mean_absolute_error(actuals, predictions)
    mse = mean_squared_error(actuals, predictions)
    rmse = np.sqrt(mse)

    # Percentage-based metrics
    mape = mean_absolute_percentage_error(actuals, predictions)

    # Handle R² calculation safely
    try:
        r2 = r2_score(actuals, predictions)
    except:
        r2 = float('nan')

    # Symmetric Mean Absolute Percentage Error (SMAPE)
    smape = 100 * np.mean(2 * np.abs(predictions - actuals) / (np.abs(actuals) + np.abs(predictions)))

    # Mean Directional Accuracy (MDA)
    if len(actuals) > 1:
        actual_direction = np.diff(actuals) > 0
        pred_direction = np.diff(predictions) > 0
        mda = np.mean(actual_direction == pred_direction) * 100
    else:
        mda = float('nan')

    # Tracking Signal (TS)
    cumulative_error = np.sum(predictions - actuals)
    mad = np.mean(np.abs(predictions - actuals))
    ts = cumulative_error / mad if mad != 0 else float('nan')

    # Normalized metrics
    actual_std = np.std(actuals)
    nrmse = rmse / actual_std if actual_std != 0 else float('nan')
    nmae = mae / np.mean(np.abs(actuals)) if np.mean(np.abs(actuals)) != 0 else float('nan')

    # Forecast bias
    bias = np.mean(predictions - actuals)

    return {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'MAPE': mape,
        'SMAPE': smape,
        'R2': r2,
        'MDA': mda,
        'Tracking_Signal': ts,
        'NRMSE': nrmse,
        'NMAE': nmae,
        'Bias': bias,
        'Mean_Actual': np.mean(actuals),
        'Mean_Predicted': np.mean(predictions),
        'Std_Actual': np.std(actuals),
        'Std_Predicted': np.std(predictions)
    }

def train_nbeats(model, train_loader, val_loader, epochs, lr, device):
    """
    Enhanced training loop with comprehensive logging to MLflow, DagsHub, and Wandb.
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    train_losses = []
    val_losses = []

    # Watch model with wandb
    wandb.watch(model, log_freq=100)

    for epoch in range(1, epochs + 1):
        model.train()
        running_train_loss = 0.0
        batch_count = 0

        # Training loop
        for batch_idx, (backcast, forecast) in enumerate(train_loader):
            # Move tensors to the correct device
            backcast, forecast = backcast.to(device), forecast.to(device)

            optimizer.zero_grad()
            outputs = model(backcast)
            loss = criterion(outputs, forecast)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item()
            batch_count += 1

            # Log batch-level metrics to wandb
            if batch_idx % 10 == 0:  # Log every 10 batches
                wandb.log({
                    "batch_train_loss": loss.item(),
                    "epoch": epoch,
                    "batch": batch_idx
                })

        avg_train_loss = running_train_loss / max(1, len(train_loader))
        train_losses.append(avg_train_loss)

        # Validation loop
        model.eval()
        running_val_loss = 0.0

        with torch.no_grad():
            for backcast, forecast in val_loader:
                # Move tensors to the correct device
                backcast, forecast = backcast.to(device), forecast.to(device)

                outputs = model(backcast)
                loss = criterion(outputs, forecast)
                running_val_loss += loss.item()

        avg_val_loss = running_val_loss / max(1, len(val_loader))
        val_losses.append(avg_val_loss)

        # Log epoch-level metrics to all platforms
        metrics = {
            "epoch": epoch,
            "train_loss": avg_train_loss,
            "val_loss": avg_val_loss,
            "learning_rate": lr
        }

        # Log to MLflow
        mlflow.log_metrics(metrics, step=epoch)

        # Log to Wandb
        wandb.log(metrics)

        # Enhanced periodic logging
        if epoch % 10 == 0 or epoch == 1 or epoch == epochs:
            print(f"[Epoch {epoch}/{epochs}] Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f}")

            # Calculate and log additional training metrics
            overfitting_gap = avg_val_loss - avg_train_loss
            improvement_rate = (train_losses[0] - avg_train_loss) / train_losses[0] * 100 if epoch > 1 else 0

            additional_metrics = {
                "overfitting_gap": overfitting_gap,
                "training_improvement_rate": improvement_rate,
                "epoch_batch_count": batch_count
            }

            mlflow.log_metrics(additional_metrics, step=epoch)
            wandb.log(additional_metrics)

    return train_losses, val_losses

def create_forecast_plots(actuals, predictions, sales_mean, sales_std):
    """Create and save forecast visualization plots with enhanced logging"""

    # Create forecast comparison plot
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # Plot 1: Actual vs Predicted
    axes[0, 0].plot(actuals, label='Actual', color='blue', linewidth=2)
    axes[0, 0].plot(predictions, label='Predicted', color='red', linewidth=2, linestyle='--')
    axes[0, 0].set_title('Actual vs Predicted Sales')
    axes[0, 0].set_xlabel('Time Steps')
    axes[0, 0].set_ylabel('Sales')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Plot 2: Residuals
    residuals = np.array(predictions) - np.array(actuals)
    axes[0, 1].plot(residuals, color='green', linewidth=1)
    axes[0, 1].axhline(y=0, color='black', linestyle='-', alpha=0.5)
    axes[0, 1].set_title('Prediction Residuals')
    axes[0, 1].set_xlabel('Time Steps')
    axes[0, 1].set_ylabel('Residuals (Predicted - Actual)')
    axes[0, 1].grid(True, alpha=0.3)

    # Plot 3: Scatter plot
    axes[1, 0].scatter(actuals, predictions, alpha=0.6, color='purple')
    min_val = min(min(actuals), min(predictions))
    max_val = max(max(actuals), max(predictions))
    axes[1, 0].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2)
    axes[1, 0].set_xlabel('Actual Sales')
    axes[1, 0].set_ylabel('Predicted Sales')
    axes[1, 0].set_title('Actual vs Predicted Scatter Plot')
    axes[1, 0].grid(True, alpha=0.3)

    # Plot 4: Distribution comparison
    axes[1, 1].hist(actuals, bins=20, alpha=0.6, label='Actual', color='blue', density=True)
    axes[1, 1].hist(predictions, bins=20, alpha=0.6, label='Predicted', color='red', density=True)
    axes[1, 1].set_xlabel('Sales Values')
    axes[1, 1].set_ylabel('Density')
    axes[1, 1].set_title('Distribution Comparison')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('forecast_evaluation.png', dpi=300, bbox_inches='tight')

    # Log to all platforms
    mlflow.log_artifact('forecast_evaluation.png')
    wandb.log({"forecast_evaluation": wandb.Image('forecast_evaluation.png')})

    plt.show()

def log_training_history(train_losses, val_losses):
    """Enhanced training history logging"""

    # Create training history plot
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Plot 1: Loss curves
    epochs = range(1, len(train_losses) + 1)
    axes[0].plot(epochs, train_losses, 'b-', label='Training Loss', linewidth=2)
    axes[0].plot(epochs, val_losses, 'r-', label='Validation Loss', linewidth=2)
    axes[0].set_title('Training and Validation Loss')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Plot 2: Loss curves (log scale)
    axes[1].semilogy(epochs, train_losses, 'b-', label='Training Loss', linewidth=2)
    axes[1].semilogy(epochs, val_losses, 'r-', label='Validation Loss', linewidth=2)
    axes[1].set_title('Training and Validation Loss (Log Scale)')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Log Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('training_history.png', dpi=300, bbox_inches='tight')

    # Log to all platforms
    mlflow.log_artifact('training_history.png')
    wandb.log({"training_history": wandb.Image('training_history.png')})

    plt.show()

    # Enhanced metrics logging
    final_metrics = {
        "final_train_loss": train_losses[-1],
        "final_val_loss": val_losses[-1],
        "min_val_loss": min(val_losses),
        "min_val_loss_epoch": val_losses.index(min(val_losses)) + 1,
        "total_epochs": len(train_losses)
    }

    # Calculate learning curve metrics
    early_train_loss = np.mean(train_losses[:10]) if len(train_losses) >= 10 else train_losses[0]
    late_train_loss = np.mean(train_losses[-10:]) if len(train_losses) >= 10 else train_losses[-1]
    train_improvement = (early_train_loss - late_train_loss) / early_train_loss * 100

    early_val_loss = np.mean(val_losses[:10]) if len(val_losses) >= 10 else val_losses[0]
    late_val_loss = np.mean(val_losses[-10:]) if len(val_losses) >= 10 else val_losses[-1]
    val_improvement = (early_val_loss - late_val_loss) / early_val_loss * 100

    final_metrics.update({
        "train_improvement_percent": train_improvement,
        "val_improvement_percent": val_improvement,
        "final_train_val_gap": val_losses[-1] - train_losses[-1],
        "overfitting_ratio": val_losses[-1] / train_losses[-1],
        "convergence_stability": np.std(val_losses[-10:]) if len(val_losses) >= 10 else 0
    })

    # Log to all platforms
    mlflow.log_metrics(final_metrics)
    wandb.log(final_metrics)

def run_nbeats_experiment(dagshub_repo_owner="your-username", dagshub_repo_name="your-repo-name"):
    """Enhanced N-BEATS experiment runner with comprehensive tracking"""

    # Setup all tracking systems
    setup_tracking(
        project_name="walmart-nbeats-forecasting",
        dagshub_repo_owner=dagshub_repo_owner,
        dagshub_repo_name=dagshub_repo_name
    )

    # Check if CUDA is available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Load data with error handling
    try:
        df = pd.read_csv('train.csv')
        df['Date'] = pd.to_datetime(df['Date'])
        print(f"Data loaded successfully. Shape: {df.shape}")
    except FileNotFoundError:
        print("Error: train.csv file not found. Please ensure the file exists in the current directory.")
        return None, None, None
    except Exception as e:
        print(f"Error loading data: {e}")
        return None, None, None

    # MLflow experiment setup
    mlflow.set_experiment("Walmart_NBEATS_Forecasting")

    with mlflow.start_run():
        # Parameters
        BACKCAST_LENGTH = 20
        FORECAST_LENGTH = 4
        BATCH_SIZE = 16
        EPOCHS = 100
        LEARNING_RATE = 0.001

        # Enhanced parameter logging
        params = {
            "backcast_length": BACKCAST_LENGTH,
            "forecast_length": FORECAST_LENGTH,
            "batch_size": BATCH_SIZE,
            "epochs": EPOCHS,
            "learning_rate": LEARNING_RATE,
            "device": str(device),
            "optimizer": "Adam",
            "criterion": "MSELoss",
            "data_source": "walmart_sales",
            "model_architecture": "NBEATS"
        }

        # Log to all platforms
        mlflow.log_params(params)
        wandb.config.update(params)

        # Prepare dataset
        weekly_sales = df.groupby('Date')['Weekly_Sales'].sum().values
        print(f"Weekly sales data points: {len(weekly_sales)}")

        # Data validation
        min_required = BACKCAST_LENGTH + FORECAST_LENGTH
        if len(weekly_sales) < min_required:
            print(f"Error: Not enough data points. Need at least {min_required}, got {len(weekly_sales)}")
            return None, None, None

        # Normalize data
        sales_mean = weekly_sales.mean()
        sales_std = weekly_sales.std()
        weekly_sales_normalized = (weekly_sales - sales_mean) / sales_std

        # Log data statistics
        data_stats = {
            "sales_mean": sales_mean,
            "sales_std": sales_std,
            "data_points": len(weekly_sales),
            "sales_min": weekly_sales.min(),
            "sales_max": weekly_sales.max(),
            "sales_median": np.median(weekly_sales)
        }

        mlflow.log_params(data_stats)
        wandb.config.update(data_stats)

        # Enhanced data splitting
        train_size = int(0.7 * len(weekly_sales_normalized))
        val_size = int(0.15 * len(weekly_sales_normalized))

        train_data = weekly_sales_normalized[:train_size]
        val_data = weekly_sales_normalized[train_size:train_size + val_size]
        test_data = weekly_sales_normalized[train_size + val_size:]

        print(f"Train size: {len(train_data)}, Val size: {len(val_data)}, Test size: {len(test_data)}")

        # Data split validation and adjustment
        if len(train_data) < min_required or len(val_data) < min_required:
            print("Adjusting split for limited data...")
            train_end = int(0.6 * len(weekly_sales_normalized))
            val_end = int(0.8 * len(weekly_sales_normalized))

            train_data = weekly_sales_normalized[:train_end]
            val_data = weekly_sales_normalized[train_end:val_end]
            test_data = weekly_sales_normalized[val_end:]

            print(f"Adjusted split - Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_data)}")

        # Final validation
        if len(train_data) < min_required or len(val_data) < min_required:
            print(f"Error: Insufficient data after adjustment")
            return None, None, None

        # Create datasets and loaders
        train_dataset = WalmartDataset(train_data, BACKCAST_LENGTH, FORECAST_LENGTH)
        val_dataset = WalmartDataset(val_data, BACKCAST_LENGTH, FORECAST_LENGTH)

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # Model initialization
        model = NBEATS(
            backcast_length=BACKCAST_LENGTH,
            forecast_length=FORECAST_LENGTH,
            stack_types=['trend', 'seasonality', 'generic'],
            nb_blocks_per_stack=3,
            hidden_layer_units=128
        ).to(device)

        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

        model_info = {
            "total_parameters": total_params,
            "trainable_parameters": trainable_params,
            "model_size_mb": total_params * 4 / 1024 / 1024  # Approximate size in MB
        }

        mlflow.log_params(model_info)
        wandb.config.update(model_info)

        print(f"Model initialized with {total_params} parameters")

        # Enhanced training with comprehensive logging
        train_losses, val_losses = train_nbeats(model, train_loader, val_loader, EPOCHS, LEARNING_RATE, device)

        # Log training history and create visualizations
        log_training_history(train_losses, val_losses)

        # Save model to all platforms
        mlflow.pytorch.log_model(model, "nbeats_model")

        # Save model for wandb
        torch.save(model.state_dict(), "nbeats_model.pth")
        wandb.save("nbeats_model.pth")

        # Enhanced testing and evaluation
        if len(test_data) >= BACKCAST_LENGTH + FORECAST_LENGTH:
            test_dataset = WalmartDataset(test_data, BACKCAST_LENGTH, FORECAST_LENGTH)
            test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

            model.eval()
            predictions = []
            actuals = []

            with torch.no_grad():
                for backcast, forecast in test_loader:
                    backcast, forecast = backcast.to(device), forecast.to(device)
                    pred = model(backcast)

                    # Denormalize predictions and actuals
                    pred_denorm = pred.cpu().numpy() * sales_std + sales_mean
                    actual_denorm = forecast.cpu().numpy() * sales_std + sales_mean

                    predictions.extend(pred_denorm.flatten())
                    actuals.extend(actual_denorm.flatten())

            # Calculate comprehensive metrics
            metrics = calculate_metrics(actuals, predictions)

            # Log all metrics to all platforms
            test_metrics = {}
            for metric_name, metric_value in metrics.items():
                if not np.isnan(metric_value):
                    test_metric_name = f"test_{metric_name.lower()}"
                    test_metrics[test_metric_name] = metric_value

            mlflow.log_metrics(test_metrics)
            wandb.log(test_metrics)

            # Create summary table for wandb
            results_table = wandb.Table(
                columns=["Metric", "Value", "Description"],
                data=[
                    ["MAE", f"{metrics['MAE']:.2f}", "Mean Absolute Error"],
                    ["RMSE", f"{metrics['RMSE']:.2f}", "Root Mean Square Error"],
                    ["MAPE", f"{metrics['MAPE']:.2f}%", "Mean Absolute Percentage Error"],
                    ["SMAPE", f"{metrics['SMAPE']:.2f}%", "Symmetric MAPE"],
                    ["R²", f"{metrics['R2']:.4f}", "Coefficient of Determination"],
                    ["MDA", f"{metrics['MDA']:.2f}%", "Mean Directional Accuracy"],
                    ["Bias", f"{metrics['Bias']:.2f}", "Forecast Bias"],
                    ["NRMSE", f"{metrics['NRMSE']:.4f}", "Normalized RMSE"],
                    ["Tracking Signal", f"{metrics['Tracking_Signal']:.4f}", "Cumulative Error / MAD"]
                ]
            )
            wandb.log({"test_metrics_summary": results_table})

            # Print comprehensive results
            print(f"\n{'='*50}")
            print(f"🎯 TEST SET EVALUATION RESULTS")
            print(f"{'='*50}")
            print(f"📊 Accuracy Metrics:")
            print(f"   • MAE (Mean Absolute Error): {metrics['MAE']:.2f}")
            print(f"   • RMSE (Root Mean Square Error): {metrics['RMSE']:.2f}")
            print(f"   • R² (Coefficient of Determination): {metrics['R2']:.4f}")
            print(f"\n📈 Percentage-based Metrics:")
            print(f"   • MAPE (Mean Absolute Percentage Error): {metrics['MAPE']:.2f}%")
            print(f"   • SMAPE (Symmetric MAPE): {metrics['SMAPE']:.2f}%")
            print(f"   • MDA (Mean Directional Accuracy): {metrics['MDA']:.2f}%")
            print(f"\n⚖️ Bias and Reliability:")
            print(f"   • Forecast Bias: {metrics['Bias']:.2f}")
            print(f"   • Tracking Signal: {metrics['Tracking_Signal']:.4f}")
            print(f"   • NRMSE (Normalized RMSE): {metrics['NRMSE']:.4f}")
            print(f"   • NMAE (Normalized MAE): {metrics['NMAE']:.4f}")
            print(f"\n📋 Data Statistics:")
            print(f"   • Mean Actual Sales: {metrics['Mean_Actual']:.2f}")
            print(f"   • Mean Predicted Sales: {metrics['Mean_Predicted']:.2f}")
            print(f"   • Std Actual Sales: {metrics['Std_Actual']:.2f}")
            print(f"   • Std Predicted Sales: {metrics['Std_Predicted']:.2f}")
            print(f"{'='*50}")

            # Create and log comprehensive visualizations
            create_forecast_plots(actuals, predictions, sales_mean, sales_std)

            # Create additional analysis plots
            create_advanced_analysis_plots(actuals, predictions, train_losses, val_losses)

            # Log prediction vs actual data as wandb table
            prediction_table = wandb.Table(
                columns=["Time_Step", "Actual", "Predicted", "Error", "Abs_Error", "Pct_Error"],
                data=[
                    [i, actual, pred, pred-actual, abs(pred-actual), abs(pred-actual)/actual*100 if actual != 0 else 0]
                    for i, (actual, pred) in enumerate(zip(actuals, predictions))
                ]
            )
            wandb.log({"predictions_detailed": prediction_table})

            return model, predictions, actuals
        else:
            print("❌ Not enough test data for evaluation")
            return model, [], []

def create_advanced_analysis_plots(actuals, predictions, train_losses, val_losses):
    """Create advanced analysis and diagnostic plots"""

    # Create a comprehensive analysis figure
    fig = plt.figure(figsize=(20, 15))

    # Plot 1: Training convergence analysis
    ax1 = plt.subplot(3, 3, 1)
    epochs = range(1, len(train_losses) + 1)
    plt.plot(epochs, train_losses, 'b-', label='Training Loss', linewidth=2)
    plt.plot(epochs, val_losses, 'r-', label='Validation Loss', linewidth=2)
    plt.fill_between(epochs, train_losses, val_losses, alpha=0.2, color='gray')
    plt.title('Training Convergence Analysis')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Plot 2: Loss ratio analysis
    ax2 = plt.subplot(3, 3, 2)
    loss_ratios = [v/t if t != 0 else 1 for v, t in zip(val_losses, train_losses)]
    plt.plot(epochs, loss_ratios, 'g-', linewidth=2)
    plt.axhline(y=1, color='black', linestyle='--', alpha=0.5, label='Perfect Fit')
    plt.title('Validation/Training Loss Ratio')
    plt.xlabel('Epochs')
    plt.ylabel('Val Loss / Train Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Plot 3: Error distribution
    ax3 = plt.subplot(3, 3, 3)
    errors = np.array(predictions) - np.array(actuals)
    plt.hist(errors, bins=20, alpha=0.7, color='purple', density=True, edgecolor='black')
    plt.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero Error')
    plt.axvline(x=np.mean(errors), color='orange', linestyle='-', linewidth=2, label=f'Mean Error: {np.mean(errors):.2f}')
    plt.title('Error Distribution')
    plt.xlabel('Prediction Error')
    plt.ylabel('Density')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Plot 4: Q-Q plot for error normality
    from scipy import stats
    ax4 = plt.subplot(3, 3, 4)
    stats.probplot(errors, dist="norm", plot=plt)
    plt.title('Q-Q Plot: Error Normality Check')
    plt.grid(True, alpha=0.3)

    # Plot 5: Cumulative error
    ax5 = plt.subplot(3, 3, 5)
    cumsum_errors = np.cumsum(errors)
    plt.plot(range(len(cumsum_errors)), cumsum_errors, 'r-', linewidth=2)
    plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    plt.title('Cumulative Prediction Error')
    plt.xlabel('Time Steps')
    plt.ylabel('Cumulative Error')
    plt.grid(True, alpha=0.3)

    # Plot 6: Actual vs Predicted with confidence intervals
    ax6 = plt.subplot(3, 3, 6)
    time_steps = range(len(actuals))
    plt.plot(time_steps, actuals, 'b-', label='Actual', linewidth=2, marker='o', markersize=4)
    plt.plot(time_steps, predictions, 'r--', label='Predicted', linewidth=2, marker='s', markersize=4)

    # Add confidence intervals (using standard error)
    std_error = np.std(errors)
    upper_bound = np.array(predictions) + 1.96 * std_error
    lower_bound = np.array(predictions) - 1.96 * std_error
    plt.fill_between(time_steps, lower_bound, upper_bound, alpha=0.2, color='red', label='95% CI')

    plt.title('Predictions with Confidence Intervals')
    plt.xlabel('Time Steps')
    plt.ylabel('Sales')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Plot 7: Percentage error over time
    ax7 = plt.subplot(3, 3, 7)
    pct_errors = [abs(p-a)/a*100 if a != 0 else 0 for p, a in zip(predictions, actuals)]
    plt.plot(range(len(pct_errors)), pct_errors, 'g-', linewidth=2, marker='o', markersize=3)
    plt.axhline(y=np.mean(pct_errors), color='red', linestyle='--', label=f'Mean: {np.mean(pct_errors):.1f}%')
    plt.title('Absolute Percentage Error Over Time')
    plt.xlabel('Time Steps')
    plt.ylabel('Absolute Percentage Error (%)')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Plot 8: Autocorrelation of errors
    ax8 = plt.subplot(3, 3, 8)
    from statsmodels.tsa.stattools import acf
    if len(errors) > 10:  # Need sufficient data for autocorrelation
        lags = min(20, len(errors)//2)
        autocorr = acf(errors, nlags=lags, fft=True)
        plt.bar(range(len(autocorr)), autocorr, alpha=0.7, color='orange')
        plt.axhline(y=0, color='black', linestyle='-', alpha=0.5)
        plt.axhline(y=0.2, color='red', linestyle='--', alpha=0.5, label='Significance')
        plt.axhline(y=-0.2, color='red', linestyle='--', alpha=0.5)
    plt.title('Error Autocorrelation')
    plt.xlabel('Lag')
    plt.ylabel('Autocorrelation')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Plot 9: Model performance summary
    ax9 = plt.subplot(3, 3, 9)
    metrics_names = ['MAE', 'RMSE', 'MAPE', 'SMAPE', 'R²']
    metrics_values = [
        mean_absolute_error(actuals, predictions),
        np.sqrt(mean_squared_error(actuals, predictions)),
        mean_absolute_percentage_error(actuals, predictions),
        100 * np.mean(2 * np.abs(np.array(predictions) - np.array(actuals)) /
                     (np.abs(actuals) + np.abs(predictions))),
        r2_score(actuals, predictions) if len(set(actuals)) > 1 else 0
    ]

    colors = ['skyblue', 'lightcoral', 'lightgreen', 'gold', 'plum']
    bars = plt.bar(metrics_names, metrics_values, color=colors, alpha=0.7, edgecolor='black')
    plt.title('Model Performance Summary')
    plt.ylabel('Metric Value')
    plt.xticks(rotation=45)

    # Add value labels on bars
    for bar, value in zip(bars, metrics_values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(metrics_values)*0.01,
                f'{value:.2f}', ha='center', va='bottom', fontweight='bold')

    plt.grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    plt.savefig('advanced_analysis.png', dpi=300, bbox_inches='tight')

    # Log to all platforms
    mlflow.log_artifact('advanced_analysis.png')
    wandb.log({"advanced_analysis": wandb.Image('advanced_analysis.png')})

    plt.show()

def create_model_interpretation_plots(model, sample_input, device):
    """Create model interpretation and feature importance plots"""

    try:
        model.eval()
        sample_input = sample_input.to(device)

        # Get model predictions and intermediate outputs
        with torch.no_grad():
            prediction = model(sample_input)

        fig, axes = plt.subplots(2, 2, figsize=(15, 10))

        # Plot 1: Input vs Prediction
        axes[0, 0].plot(sample_input[0].cpu().numpy(), 'b-', label='Input (Backcast)', linewidth=2)
        axes[0, 0].set_title('Model Input (Historical Data)')
        axes[0, 0].set_xlabel('Time Steps')
        axes[0, 0].set_ylabel('Normalized Sales')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)

        # Plot 2: Prediction output
        axes[0, 1].plot(prediction[0].cpu().numpy(), 'r-', label='Prediction (Forecast)', linewidth=2, marker='o')
        axes[0, 1].set_title('Model Prediction (Forecast)')
        axes[0, 1].set_xlabel('Future Time Steps')
        axes[0, 1].set_ylabel('Normalized Sales')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)

        # Plot 3: Combined view
        input_extended = list(sample_input[0].cpu().numpy()) + [None] * len(prediction[0])
        pred_extended = [None] * len(sample_input[0]) + list(prediction[0].cpu().numpy())

        axes[1, 0].plot(range(len(input_extended)), input_extended, 'b-', label='Historical', linewidth=2)
        axes[1, 0].plot(range(len(pred_extended)), pred_extended, 'r-', label='Forecast', linewidth=2, marker='o')
        axes[1, 0].axvline(x=len(sample_input[0])-1, color='gray', linestyle='--', alpha=0.7, label='Forecast Start')
        axes[1, 0].set_title('Historical Data → Forecast')
        axes[1, 0].set_xlabel('Time Steps')
        axes[1, 0].set_ylabel('Normalized Sales')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)

        # Plot 4: Model architecture summary (text-based)
        axes[1, 1].text(0.1, 0.9, "N-BEATS Model Architecture:", fontsize=14, fontweight='bold', transform=axes[1, 1].transAxes)
        axes[1, 1].text(0.1, 0.8, f"• Backcast Length: {model.backcast_length}", fontsize=12, transform=axes[1, 1].transAxes)
        axes[1, 1].text(0.1, 0.7, f"• Forecast Length: {model.forecast_length}", fontsize=12, transform=axes[1, 1].transAxes)
        axes[1, 1].text(0.1, 0.6, f"• Stack Types: {model.stack_types}", fontsize=12, transform=axes[1, 1].transAxes)
        axes[1, 1].text(0.1, 0.5, f"• Blocks per Stack: {model.nb_blocks_per_stack}", fontsize=12, transform=axes[1, 1].transAxes)
        axes[1, 1].text(0.1, 0.4, f"• Hidden Units: {model.hidden_layer_units}", fontsize=12, transform=axes[1, 1].transAxes)

        total_params = sum(p.numel() for p in model.parameters())
        axes[1, 1].text(0.1, 0.3, f"• Total Parameters: {total_params:,}", fontsize=12, transform=axes[1, 1].transAxes)
        axes[1, 1].text(0.1, 0.2, f"• Model Size: ~{total_params * 4 / 1024 / 1024:.1f} MB", fontsize=12, transform=axes[1, 1].transAxes)

        axes[1, 1].set_xlim(0, 1)
        axes[1, 1].set_ylim(0, 1)
        axes[1, 1].axis('off')

        plt.tight_layout()
        plt.savefig('model_interpretation.png', dpi=300, bbox_inches='tight')

        # Log to all platforms
        mlflow.log_artifact('model_interpretation.png')
        wandb.log({"model_interpretation": wandb.Image('model_interpretation.png')})

        plt.show()

    except Exception as e:
        print(f"Could not create model interpretation plots: {e}")

# Usage example and main execution
if __name__ == "__main__":
    print("🚀 Starting Enhanced N-BEATS Experiment with MLflow, DagsHub & Wandb Integration")
    print("="*80)

    # Configure your credentials here
    DAGSHUB_REPO_OWNER = "ekvirika"  # Replace with your DagsHub username
    DAGSHUB_REPO_NAME = "WalmartRecruiting"   # Replace with your DagsHub repository name

    try:
        # Run the experiment
        model, predictions, actuals = run_nbeats_experiment(
            dagshub_repo_owner=DAGSHUB_REPO_OWNER,
            dagshub_repo_name=DAGSHUB_REPO_NAME
        )

        if model is not None:
            print("\n✅ Experiment completed successfully!")
            print(f"📊 Results logged to:")
            print(f"   • MLflow: Check your MLflow UI")
            print(f"   • DagsHub: https://dagshub.com/{DAGSHUB_REPO_OWNER}/{DAGSHUB_REPO_NAME}")
            print(f"   • Wandb: Check your Wandb dashboard")

            # Create model interpretation plots if we have predictions
            if len(predictions) > 0:
                # Create a sample input for interpretation
                try:
                    df = pd.read_csv('train.csv')
                    weekly_sales = df.groupby('Date')['Weekly_Sales'].sum().values
                    sales_mean = weekly_sales.mean()
                    sales_std = weekly_sales.std()
                    weekly_sales_normalized = (weekly_sales - sales_mean) / sales_std

                    # Get a sample input
                    sample_input = torch.FloatTensor(weekly_sales_normalized[:20]).unsqueeze(0)
                    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

                    create_model_interpretation_plots(model, sample_input, device)

                except Exception as e:
                    print(f"⚠️ Could not create interpretation plots: {e}")

        else:
            print("❌ Experiment failed. Please check the error messages above.")

    except Exception as e:
        print(f"❌ Experiment failed with error: {e}")

    finally:
        # Clean up wandb
        try:
            wandb.finish()
        except:
            pass

# Additional utility functions for experiment management

def compare_experiments(experiment_names, metric='test_mae'):
    """
    Compare multiple experiments from MLflow

    Args:
        experiment_names: List of experiment names to compare
        metric: Metric to compare (default: 'test_mae')
    """

    try:
        import mlflow
        from mlflow.tracking import MlflowClient

        client = MlflowClient()

        comparison_data = []

        for exp_name in experiment_names:
            experiment = client.get_experiment_by_name(exp_name)
            if experiment:
                runs = client.search_runs(experiment_ids=[experiment.experiment_id])

                for run in runs:
                    if metric in run.data.metrics:
                        comparison_data.append({
                            'experiment': exp_name,
                            'run_id': run.info.run_id,
                            'metric_value': run.data.metrics[metric],
                            'start_time': run.info.start_time
                        })

        # Create comparison DataFrame
        import pandas as pd
        df_comparison = pd.DataFrame(comparison_data)

        if not df_comparison.empty:
            print("🔍 Experiment Comparison Results:")
            print("="*50)
            summary = df_comparison.groupby('experiment')[['metric_value']].agg(['mean', 'std', 'min', 'max'])
            print(summary)

            # Create comparison plot
            plt.figure(figsize=(12, 6))
            df_comparison.boxplot(column='metric_value', by='experiment')
            plt.title(f'Experiment Comparison: {metric}')
            plt.ylabel(metric)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig('experiment_comparison.png', dpi=300, bbox_inches='tight')
            plt.show()

            return df_comparison
        else:
            print("No comparison data found.")
            return None

    except Exception as e:
        print(f"Error comparing experiments: {e}")
        return None

def export_experiment_report(run_id, output_file='experiment_report.html'):
    """
    Export a comprehensive HTML report for a specific experiment run

    Args:
        run_id: MLflow run ID
        output_file: Output HTML file name
    """

    try:
        from mlflow.tracking import MlflowClient
        import json

        client = MlflowClient()
        run = client.get_run(run_id)

        # Generate HTML report
        html_content = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>N-BEATS Experiment Report</title>
            <style>
                body {{ font-family: Arial, sans-serif; margin: 40px; }}
                .header {{ background-color: #f0f0f0; padding: 20px; border-radius: 10px; }}
                .section {{ margin: 20px 0; }}
                .metric {{ background-color: #e8f4f8; padding: 10px; margin: 5px 0; border-radius: 5px; }}
                table {{ border-collapse: collapse; width: 100%; }}
                th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
                th {{ background-color: #f2f2f2; }}
            </style>
        </head>
        <body>
            <div class="header">
                <h1>🎯 N-BEATS Experiment Report</h1>
                <p><strong>Run ID:</strong> {run_id}</p>
                <p><strong>Start Time:</strong> {run.info.start_time}</p>
                <p><strong>Status:</strong> {run.info.status}</p>
            </div>

            <div class="section">
                <h2>📋 Parameters</h2>
                <table>
                    <tr><th>Parameter</th><th>Value</th></tr>
        """

        for key, value in run.data.params.items():
            html_content += f"<tr><td>{key}</td><td>{value}</td></tr>"

        html_content += """
                </table>
            </div>

            <div class="section">
                <h2>📊 Metrics</h2>
        """

        for key, value in run.data.metrics.items():
            html_content += f'<div class="metric"><strong>{key}:</strong> {value:.6f}</div>'

        html_content += """
            </div>

            <div class="section">
                <h2>📁 Artifacts</h2>
                <ul>
        """

        artifacts = client.list_artifacts(run_id)
        for artifact in artifacts:
            html_content += f"<li>{artifact.path}</li>"

        html_content += """
                </ul>
            </div>
        </body>
        </html>
        """

        with open(output_file, 'w') as f:
            f.write(html_content)

        print(f"✅ Experiment report exported to: {output_file}")

    except Exception as e:
        print(f"❌ Error exporting report: {e}")

print("📚 Enhanced N-BEATS experiment script loaded successfully!")
print("🔧 Available functions:")
print("   • run_nbeats_experiment() - Main experiment runner")
print("   • compare_experiments() - Compare multiple experiments")
print("   • export_experiment_report() - Export detailed HTML report")
print("   • setup_tracking() - Initialize all tracking systems")
print("\n💡 Don't forget to:")
print("   1. Replace DAGSHUB_REPO_OWNER and DAGSHUB_REPO_NAME with your actual values")
print("   2. Ensure you have train.csv in your working directory")
print("   3. Install required packages: mlflow, wandb, dagshub, torch, sklearn, matplotlib, scipy, statsmodels")

🚀 Starting Enhanced N-BEATS Experiment with MLflow, DagsHub & Wandb Integration


Initialized MLflow to track repo "ekvirika/WalmartRecruiting"

Repository ekvirika/WalmartRecruiting initialized!

❌ Experiment failed with error: module 'dagshub.auth' has no attribute 'get_username'
📚 Enhanced N-BEATS experiment script loaded successfully!
🔧 Available functions:
   • run_nbeats_experiment() - Main experiment runner
   • compare_experiments() - Compare multiple experiments
   • export_experiment_report() - Export detailed HTML report
   • setup_tracking() - Initialize all tracking systems

💡 Don't forget to:
   1. Replace DAGSHUB_REPO_OWNER and DAGSHUB_REPO_NAME with your actual values
   2. Ensure you have train.csv in your working directory
   3. Install required packages: mlflow, wandb, dagshub, torch, sklearn, matplotlib, scipy, statsmodels


In [14]:
!pip install wand dagshub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 7.6 MB/s eta 0:00:00
